# Strategy Pattern Tutorial 🎯

## Table of Contents
1. [What is the Strategy Pattern?](#what-is-the-strategy-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Dynamic Strategy Selection (Advanced)](#dynamic-strategy-selection-advanced)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## What is the Strategy Pattern?

The **Strategy Pattern** defines a family of algorithms, encapsulates each one, and makes them interchangeable. It lets the algorithm vary independently from clients that use it.

### Real-world Analogy
Think of **navigation apps** like Google Maps. When you want to go somewhere, you can choose different strategies:
- 🚗 **Fastest route** - Takes highways, ignores traffic
- 🛣️ **Avoid tolls** - Takes longer but saves money
- 🚶 **Walking directions** - Pedestrian paths only
- 🚲 **Bike-friendly** - Uses bike lanes

The app doesn't change, but the **strategy** for finding a route changes based on your choice.

### Key Points
- ✅ **Encapsulates algorithms** in separate classes
- ✅ **Interchangeable at runtime** - can switch strategies dynamically
- ✅ **Eliminates conditionals** - no more huge if/else chains
- ✅ **Easy to extend** - add new strategies without changing existing code

## Why Do We Need It?

Let's see a problem that the Strategy pattern solves:

In [ ]:
# 🚫 PROBLEM: Without Strategy Pattern - Messy Conditionals

class ShoppingCart:
    """Shopping cart with payment processing (problematic version)"""
    
    def __init__(self):
        self.items = []
    
    def add_item(self, name: str, price: float):
        self.items.append({"name": name, "price": price})
        print(f"Added {name} (${price}) to cart")
    
    def get_total(self) -> float:
        return sum(item["price"] for item in self.items)
    
    def checkout(self, payment_method: str, **payment_details):
        """This method becomes a nightmare with many payment options!"""
        total = self.get_total()
        print(f"\n🛒 Checkout: Total = ${total:.2f}")
        
        # PROBLEM: Huge if/else chain that grows with every new payment method!
        if payment_method == "credit_card":
            card_number = payment_details["card_number"]
            cvv = payment_details["cvv"]
            expiry = payment_details["expiry"]
            
            # Credit card processing logic
            print(f"💳 Processing credit card ending in {card_number[-4:]}")
            print(f"   Validating CVV: {cvv}")
            print(f"   Checking expiry: {expiry}")
            
            if len(card_number) != 16:
                print("❌ Invalid card number")
                return False
            
            print("✅ Credit card payment successful")
            
        elif payment_method == "paypal":
            email = payment_details["email"]
            password = payment_details["password"]
            
            # PayPal processing logic
            print(f"🅿️ Logging into PayPal with {email}")
            print(f"   Authenticating...")
            
            if "@" not in email:
                print("❌ Invalid email")
                return False
            
            print("✅ PayPal payment successful")
            
        elif payment_method == "apple_pay":
            touch_id = payment_details["touch_id"]
            device_id = payment_details["device_id"]
            
            # Apple Pay processing logic
            print(f"🍎 Authenticating with Touch ID")
            print(f"   Device ID: {device_id}")
            
            if not touch_id:
                print("❌ Touch ID failed")
                return False
            
            print("✅ Apple Pay payment successful")
            
        elif payment_method == "bitcoin":
            wallet_address = payment_details["wallet_address"]
            private_key = payment_details["private_key"]
            
            # Bitcoin processing logic
            print(f"₿ Processing Bitcoin payment")
            print(f"   From wallet: {wallet_address[:10]}...")
            print(f"   Creating transaction...")
            
            if len(wallet_address) < 26:
                print("❌ Invalid wallet address")
                return False
            
            print("✅ Bitcoin payment successful")
            
        else:
            print(f"❌ Unknown payment method: {payment_method}")
            return False
        
        print("📦 Order confirmed!")
        return True

# Test the problematic approach
print("Without Strategy Pattern:")
cart = ShoppingCart()
cart.add_item("Laptop", 999.99)
cart.add_item("Mouse", 25.99)

# Each payment method requires different parameters - confusing!
cart.checkout("credit_card", 
             card_number="1234567890123456", 
             cvv="123", 
             expiry="12/25")

print("\n❌ Problems with this approach:")
print("1. checkout() method becomes HUGE with every new payment type")
print("2. Hard to test - all payment logic mixed together")
print("3. Violates Single Responsibility Principle")
print("4. Hard to maintain - change one payment method, risk breaking others")
print("5. Can't easily add new payment methods")
print("6. Different parameter requirements for each method make API confusing")

## Simple Implementation

Now let's fix this problem with the Strategy pattern:

In [ ]:
# ✅ SOLUTION: With Strategy Pattern

from abc import ABC, abstractmethod
from typing import Dict, Any

# Strategy interface
class PaymentStrategy(ABC):
    """Interface for payment strategies"""
    
    @abstractmethod
    def pay(self, amount: float) -> bool:
        """Process payment and return success status"""
        pass
    
    @abstractmethod
    def get_payment_details(self) -> str:
        """Get human-readable payment method details"""
        pass

# Concrete strategies
class CreditCardStrategy(PaymentStrategy):
    """Credit card payment strategy"""
    
    def __init__(self, card_number: str, cvv: str, expiry: str, cardholder: str):
        self.card_number = card_number
        self.cvv = cvv
        self.expiry = expiry
        self.cardholder = cardholder
    
    def pay(self, amount: float) -> bool:
        print(f"💳 Processing credit card payment of ${amount:.2f}")
        print(f"   Card: **** **** **** {self.card_number[-4:]}")
        print(f"   Cardholder: {self.cardholder}")
        print(f"   Validating CVV and expiry...")
        
        # Validation logic
        if len(self.card_number) != 16:
            print("❌ Invalid card number")
            return False
        
        if len(self.cvv) != 3:
            print("❌ Invalid CVV")
            return False
        
        print("✅ Credit card payment successful")
        return True
    
    def get_payment_details(self) -> str:
        return f"Credit Card ending in {self.card_number[-4:]}"

class PayPalStrategy(PaymentStrategy):
    """PayPal payment strategy"""
    
    def __init__(self, email: str, password: str):
        self.email = email
        self.password = password
    
    def pay(self, amount: float) -> bool:
        print(f"🅿️ Processing PayPal payment of ${amount:.2f}")
        print(f"   Account: {self.email}")
        print(f"   Authenticating with PayPal...")
        
        # Validation logic
        if "@" not in self.email:
            print("❌ Invalid email address")
            return False
        
        if len(self.password) < 6:
            print("❌ Password too short")
            return False
        
        print("✅ PayPal payment successful")
        return True
    
    def get_payment_details(self) -> str:
        return f"PayPal ({self.email})"

class ApplePayStrategy(PaymentStrategy):
    """Apple Pay payment strategy"""
    
    def __init__(self, device_id: str, touch_id_enabled: bool = True):
        self.device_id = device_id
        self.touch_id_enabled = touch_id_enabled
    
    def pay(self, amount: float) -> bool:
        print(f"🍎 Processing Apple Pay payment of ${amount:.2f}")
        print(f"   Device: {self.device_id}")
        print(f"   Requesting biometric authentication...")
        
        if not self.touch_id_enabled:
            print("❌ Touch ID not enabled")
            return False
        
        print("   👆 Touch ID verified")
        print("✅ Apple Pay payment successful")
        return True
    
    def get_payment_details(self) -> str:
        return f"Apple Pay (Device: {self.device_id})"

class BitcoinStrategy(PaymentStrategy):
    """Bitcoin payment strategy"""
    
    def __init__(self, wallet_address: str, private_key: str):
        self.wallet_address = wallet_address
        self.private_key = private_key
    
    def pay(self, amount: float) -> bool:
        print(f"₿ Processing Bitcoin payment of ${amount:.2f}")
        print(f"   From wallet: {self.wallet_address[:10]}...{self.wallet_address[-6:]}")
        print(f"   Creating blockchain transaction...")
        
        if len(self.wallet_address) < 26:
            print("❌ Invalid wallet address")
            return False
        
        print("   📡 Broadcasting to network...")
        print("✅ Bitcoin payment successful")
        return True
    
    def get_payment_details(self) -> str:
        return f"Bitcoin Wallet ({self.wallet_address[:10]}...)"

# Context class that uses strategies
class ShoppingCart:
    """Shopping cart that can use any payment strategy"""
    
    def __init__(self):
        self.items = []
        self.payment_strategy: PaymentStrategy = None
    
    def add_item(self, name: str, price: float):
        self.items.append({"name": name, "price": price})
        print(f"Added {name} (${price:.2f}) to cart")
    
    def get_total(self) -> float:
        return sum(item["price"] for item in self.items)
    
    def set_payment_strategy(self, strategy: PaymentStrategy):
        """Change payment method - this is the key to Strategy pattern!"""
        self.payment_strategy = strategy
        print(f"💳 Payment method set to: {strategy.get_payment_details()}")
    
    def checkout(self) -> bool:
        """Simple checkout - the strategy handles payment details!"""
        if not self.payment_strategy:
            print("❌ No payment method selected")
            return False
        
        total = self.get_total()
        print(f"\n🛒 Checkout: Total = ${total:.2f}")
        print(f"   Payment method: {self.payment_strategy.get_payment_details()}")
        
        # Delegate to the strategy!
        if self.payment_strategy.pay(total):
            print("📦 Order confirmed and will be shipped!")
            return True
        else:
            print("❌ Payment failed - order cancelled")
            return False

# Test the Strategy pattern
print("With Strategy Pattern:")

# Create shopping cart
cart = ShoppingCart()
cart.add_item("Laptop", 999.99)
cart.add_item("Mouse", 25.99)
cart.add_item("Keyboard", 75.50)

print("\n--- Trying Different Payment Methods ---")

# Try credit card
credit_card = CreditCardStrategy(
    card_number="1234567890123456",
    cvv="123",
    expiry="12/25",
    cardholder="John Doe"
)
cart.set_payment_strategy(credit_card)
cart.checkout()

# Switch to PayPal
paypal = PayPalStrategy("john.doe@email.com", "password123")
cart.set_payment_strategy(paypal)
cart.checkout()

# Switch to Apple Pay
apple_pay = ApplePayStrategy("iPhone-12-Pro", touch_id_enabled=True)
cart.set_payment_strategy(apple_pay)
cart.checkout()

print("\n✅ Benefits of Strategy Pattern:")
print("1. Each payment method is a separate, focused class")
print("2. Easy to add new payment methods without changing existing code")
print("3. Can switch payment strategies at runtime")
print("4. ShoppingCart doesn't need to know payment details")
print("5. Each strategy can be tested independently")
print("6. Follows Single Responsibility and Open-Closed principles")

## Understanding the Implementation

Let's break down how the Strategy pattern works and understand the key concepts:

### Strategy Pattern Components

The Strategy pattern has three main components:

1. **Strategy Interface** - Defines the contract all strategies must follow
2. **Concrete Strategies** - Implement specific algorithms/behaviors
3. **Context** - Uses strategies and can switch between them

In [ ]:
# Let's create a detailed example to understand the flow

from abc import ABC, abstractmethod
import time

# Strategy interface for sorting algorithms
class SortingStrategy(ABC):
    """Interface for sorting algorithms"""
    
    @abstractmethod
    def sort(self, data: list) -> list:
        """Sort the data and return sorted list"""
        pass
    
    @abstractmethod
    def get_name(self) -> str:
        """Get the name of this sorting algorithm"""
        pass
    
    @abstractmethod
    def get_time_complexity(self) -> str:
        """Get the time complexity of this algorithm"""
        pass

# Concrete strategies
class BubbleSortStrategy(SortingStrategy):
    """Bubble sort - simple but slow"""
    
    def sort(self, data: list) -> list:
        print(f"   🫧 Using Bubble Sort algorithm...")
        data = data.copy()  # Don't modify original
        n = len(data)
        
        for i in range(n):
            for j in range(0, n - i - 1):
                if data[j] > data[j + 1]:
                    data[j], data[j + 1] = data[j + 1], data[j]
        
        return data
    
    def get_name(self) -> str:
        return "Bubble Sort"
    
    def get_time_complexity(self) -> str:
        return "O(n²)"

class QuickSortStrategy(SortingStrategy):
    """Quick sort - fast and efficient"""
    
    def sort(self, data: list) -> list:
        print(f"   ⚡ Using Quick Sort algorithm...")
        return self._quicksort(data.copy())
    
    def _quicksort(self, data: list) -> list:
        if len(data) <= 1:
            return data
        
        pivot = data[len(data) // 2]
        left = [x for x in data if x < pivot]
        middle = [x for x in data if x == pivot]
        right = [x for x in data if x > pivot]
        
        return self._quicksort(left) + middle + self._quicksort(right)
    
    def get_name(self) -> str:
        return "Quick Sort"
    
    def get_time_complexity(self) -> str:
        return "O(n log n)"

class PythonSortStrategy(SortingStrategy):
    """Python's built-in sort - optimized Timsort"""
    
    def sort(self, data: list) -> list:
        print(f"   🐍 Using Python's built-in sort (Timsort)...")
        data = data.copy()
        data.sort()
        return data
    
    def get_name(self) -> str:
        return "Python Built-in Sort (Timsort)"
    
    def get_time_complexity(self) -> str:
        return "O(n log n)"

# Context class
class DataProcessor:
    """Processes data using different sorting strategies"""
    
    def __init__(self, sorting_strategy: SortingStrategy = None):
        self.sorting_strategy = sorting_strategy
        self.data = []
    
    def add_data(self, numbers: list):
        self.data.extend(numbers)
        print(f"📊 Added {len(numbers)} numbers. Total: {len(self.data)} numbers")
    
    def set_sorting_strategy(self, strategy: SortingStrategy):
        """Change the sorting algorithm"""
        self.sorting_strategy = strategy
        print(f"🔧 Sorting strategy changed to: {strategy.get_name()}")
        print(f"   Time complexity: {strategy.get_time_complexity()}")
    
    def process_data(self) -> list:
        """Sort the data using current strategy"""
        if not self.sorting_strategy:
            print("❌ No sorting strategy set!")
            return self.data
        
        if not self.data:
            print("❌ No data to process!")
            return []
        
        print(f"\n🔄 Processing {len(self.data)} numbers with {self.sorting_strategy.get_name()}")
        print(f"   Original: {self.data[:10]}{'...' if len(self.data) > 10 else ''}")
        
        start_time = time.time()
        sorted_data = self.sorting_strategy.sort(self.data)
        end_time = time.time()
        
        processing_time = (end_time - start_time) * 1000  # Convert to milliseconds
        print(f"   Sorted: {sorted_data[:10]}{'...' if len(sorted_data) > 10 else ''}")
        print(f"   ⏱️ Processing time: {processing_time:.2f}ms")
        
        return sorted_data
    
    def benchmark_strategies(self, strategies: list):
        """Compare different strategies on the same data"""
        print(f"\n🏁 Benchmarking {len(strategies)} strategies on {len(self.data)} numbers")
        results = []
        
        for strategy in strategies:
            print(f"\n--- Testing {strategy.get_name()} ---")
            original_strategy = self.sorting_strategy
            
            self.set_sorting_strategy(strategy)
            start_time = time.time()
            self.process_data()
            end_time = time.time()
            
            processing_time = (end_time - start_time) * 1000
            results.append({
                "strategy": strategy.get_name(),
                "time_ms": processing_time,
                "complexity": strategy.get_time_complexity()
            })
            
            self.sorting_strategy = original_strategy
        
        # Display results
        print("\n📊 Benchmark Results:")
        print("Strategy".ljust(30) + "Time".ljust(12) + "Complexity")
        print("-" * 55)
        for result in sorted(results, key=lambda x: x["time_ms"]):
            print(f"{result['strategy']:<30}{result['time_ms']:>8.2f}ms   {result['complexity']}")

# Demonstrate Strategy pattern in action
print("=== Strategy Pattern - Sorting Algorithms Demo ===")

# Create data processor
processor = DataProcessor()
processor.add_data([64, 34, 25, 12, 22, 11, 90, 88, 76, 50, 42])

# Create different strategies
bubble_sort = BubbleSortStrategy()
quick_sort = QuickSortStrategy()
python_sort = PythonSortStrategy()

print("\n--- Trying Different Sorting Strategies ---")

# Try bubble sort
processor.set_sorting_strategy(bubble_sort)
processor.process_data()

# Switch to quick sort
processor.set_sorting_strategy(quick_sort)
processor.process_data()

# Switch to Python's built-in sort
processor.set_sorting_strategy(python_sort)
processor.process_data()

# Benchmark all strategies
processor.benchmark_strategies([bubble_sort, quick_sort, python_sort])

print("\n🎯 Key Insights:")
print("1. The DataProcessor doesn't know HOW sorting works")
print("2. Each strategy encapsulates a different sorting algorithm")
print("3. We can switch strategies at runtime")
print("4. Easy to add new sorting algorithms")
print("5. Each strategy can be tested and optimized independently")

### Strategy Selection Based on Context

Sometimes you want to automatically choose the best strategy based on the context or data:

In [ ]:
# Smart strategy selection based on data characteristics

class SmartDataProcessor(DataProcessor):
    """Data processor that automatically chooses the best sorting strategy"""
    
    def __init__(self):
        super().__init__()
        self.available_strategies = {
            "bubble": BubbleSortStrategy(),
            "quick": QuickSortStrategy(),
            "python": PythonSortStrategy()
        }
    
    def auto_select_strategy(self) -> SortingStrategy:
        """Automatically select the best strategy based on data size"""
        data_size = len(self.data)
        
        print(f"🤖 Auto-selecting strategy for {data_size} elements...")
        
        if data_size <= 10:
            # For small datasets, bubble sort is fine and educational
            strategy = self.available_strategies["bubble"]
            reason = "Small dataset - bubble sort for educational purposes"
        elif data_size <= 1000:
            # Medium datasets - quick sort is good
            strategy = self.available_strategies["quick"]
            reason = "Medium dataset - quick sort for good performance"
        else:
            # Large datasets - use Python's optimized sort
            strategy = self.available_strategies["python"]
            reason = "Large dataset - Python's optimized Timsort"
        
        print(f"   Selected: {strategy.get_name()}")
        print(f"   Reason: {reason}")
        
        return strategy
    
    def smart_process(self) -> list:
        """Process data with automatically selected strategy"""
        strategy = self.auto_select_strategy()
        self.set_sorting_strategy(strategy)
        return self.process_data()

# Test smart strategy selection
print("\n=== Smart Strategy Selection ===")

# Test with small dataset
small_processor = SmartDataProcessor()
small_processor.add_data([5, 2, 8, 1, 9])
small_processor.smart_process()

# Test with medium dataset
medium_processor = SmartDataProcessor()
import random
medium_data = [random.randint(1, 100) for _ in range(50)]
medium_processor.add_data(medium_data)
medium_processor.smart_process()

# Test with large dataset
large_processor = SmartDataProcessor()
large_data = [random.randint(1, 10000) for _ in range(2000)]
large_processor.add_data(large_data)
large_processor.smart_process()

print("\n💡 Smart Selection Benefits:")
print("1. Automatically chooses optimal strategy")
print("2. Users don't need to know algorithm details")
print("3. Performance is optimized for different scenarios")
print("4. Can easily adjust selection criteria")

## Dynamic Strategy Selection (Advanced)

Let's build a more sophisticated example that demonstrates strategy selection based on multiple factors:

In [ ]:
# Advanced: Image compression with multiple strategies

from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple

@dataclass
class ImageInfo:
    """Information about an image"""
    width: int
    height: int
    format: str  # 'jpeg', 'png', 'bmp', etc.
    size_mb: float
    has_transparency: bool = False
    
    @property
    def total_pixels(self) -> int:
        return self.width * self.height

# Strategy interface
class CompressionStrategy(ABC):
    """Interface for image compression strategies"""
    
    @abstractmethod
    def compress(self, image_info: ImageInfo) -> Tuple[float, int]:  # (new_size_mb, quality_score)
        """Compress image and return new size and quality score (1-100)"""
        pass
    
    @abstractmethod
    def get_name(self) -> str:
        pass
    
    @abstractmethod
    def supports_transparency(self) -> bool:
        pass
    
    @abstractmethod
    def is_lossless(self) -> bool:
        pass

# Concrete strategies
class JPEGCompressionStrategy(CompressionStrategy):
    """JPEG compression - good for photos, lossy"""
    
    def __init__(self, quality: int = 85):
        self.quality = max(1, min(100, quality))  # Clamp between 1-100
    
    def compress(self, image_info: ImageInfo) -> Tuple[float, int]:
        # JPEG compression simulation
        compression_ratio = 0.1 + (self.quality / 100 * 0.4)  # 10%-50% of original
        new_size = image_info.size_mb * compression_ratio
        
        print(f"   📸 JPEG compression (quality: {self.quality})")
        print(f"      Original: {image_info.size_mb:.2f}MB → Compressed: {new_size:.2f}MB")
        print(f"      Compression ratio: {compression_ratio*100:.1f}%")
        
        return new_size, self.quality
    
    def get_name(self) -> str:
        return f"JPEG (Quality: {self.quality})"
    
    def supports_transparency(self) -> bool:
        return False
    
    def is_lossless(self) -> bool:
        return False

class PNGCompressionStrategy(CompressionStrategy):
    """PNG compression - supports transparency, lossless"""
    
    def compress(self, image_info: ImageInfo) -> Tuple[float, int]:
        # PNG compression simulation - less compression but lossless
        compression_ratio = 0.7 if image_info.has_transparency else 0.6
        new_size = image_info.size_mb * compression_ratio
        
        print(f"   🖼️ PNG compression (lossless)")
        print(f"      Original: {image_info.size_mb:.2f}MB → Compressed: {new_size:.2f}MB")
        print(f"      Compression ratio: {compression_ratio*100:.1f}%")
        
        if image_info.has_transparency:
            print(f"      ✨ Transparency preserved")
        
        return new_size, 100  # Lossless = 100% quality
    
    def get_name(self) -> str:
        return "PNG (Lossless)"
    
    def supports_transparency(self) -> bool:
        return True
    
    def is_lossless(self) -> bool:
        return True

class WebPCompressionStrategy(CompressionStrategy):
    """WebP compression - modern format, good compression"""
    
    def __init__(self, lossless: bool = False):
        self.lossless = lossless
    
    def compress(self, image_info: ImageInfo) -> Tuple[float, int]:
        if self.lossless:
            compression_ratio = 0.5  # Better than PNG
            quality = 100
        else:
            compression_ratio = 0.08  # Very good compression
            quality = 90
        
        new_size = image_info.size_mb * compression_ratio
        
        mode = "lossless" if self.lossless else "lossy"
        print(f"   🌐 WebP compression ({mode})")
        print(f"      Original: {image_info.size_mb:.2f}MB → Compressed: {new_size:.2f}MB")
        print(f"      Compression ratio: {compression_ratio*100:.1f}%")
        
        return new_size, quality
    
    def get_name(self) -> str:
        mode = "Lossless" if self.lossless else "Lossy"
        return f"WebP ({mode})"
    
    def supports_transparency(self) -> bool:
        return True
    
    def is_lossless(self) -> bool:
        return self.lossless

# Context with intelligent strategy selection
class ImageCompressor:
    """Image compressor that intelligently selects compression strategy"""
    
    def __init__(self):
        self.strategies = {
            "jpeg_high": JPEGCompressionStrategy(quality=95),
            "jpeg_medium": JPEGCompressionStrategy(quality=75),
            "jpeg_low": JPEGCompressionStrategy(quality=50),
            "png": PNGCompressionStrategy(),
            "webp_lossless": WebPCompressionStrategy(lossless=True),
            "webp_lossy": WebPCompressionStrategy(lossless=False)
        }
        self.current_strategy = None
    
    def select_strategy(self, image_info: ImageInfo, priority: str = "balanced") -> CompressionStrategy:
        """Select best strategy based on image characteristics and priority"""
        print(f"\n🎯 Selecting compression strategy...")
        print(f"   Image: {image_info.width}x{image_info.height} {image_info.format.upper()}")
        print(f"   Size: {image_info.size_mb:.2f}MB")
        print(f"   Transparency: {'Yes' if image_info.has_transparency else 'No'}")
        print(f"   Priority: {priority}")
        
        # Rule-based strategy selection
        if image_info.has_transparency:
            if priority == "quality":
                strategy_key = "png"
                reason = "Transparency + quality priority → PNG lossless"
            elif priority == "size":
                strategy_key = "webp_lossy"
                reason = "Transparency + size priority → WebP lossy"
            else:  # balanced
                strategy_key = "webp_lossless"
                reason = "Transparency + balanced → WebP lossless"
        
        elif image_info.total_pixels > 2_000_000:  # Large image (>2MP)
            if priority == "quality":
                strategy_key = "jpeg_high"
                reason = "Large photo + quality priority → JPEG high quality"
            elif priority == "size":
                strategy_key = "webp_lossy"
                reason = "Large photo + size priority → WebP lossy"
            else:  # balanced
                strategy_key = "jpeg_medium"
                reason = "Large photo + balanced → JPEG medium quality"
        
        elif image_info.size_mb > 5.0:  # Large file size
            if priority == "quality":
                strategy_key = "webp_lossless"
                reason = "Large file + quality priority → WebP lossless"
            else:  # size or balanced
                strategy_key = "webp_lossy"
                reason = "Large file + size priority → WebP lossy"
        
        else:  # Small/medium image
            if priority == "quality":
                strategy_key = "png"
                reason = "Small image + quality priority → PNG lossless"
            elif priority == "size":
                strategy_key = "jpeg_low"
                reason = "Small image + size priority → JPEG low quality"
            else:  # balanced
                strategy_key = "jpeg_medium"
                reason = "Small image + balanced → JPEG medium quality"
        
        strategy = self.strategies[strategy_key]
        print(f"   ✅ Selected: {strategy.get_name()}")
        print(f"   📝 Reason: {reason}")
        
        return strategy
    
    def compress_image(self, image_info: ImageInfo, priority: str = "balanced") -> dict:
        """Compress image with automatically selected strategy"""
        strategy = self.select_strategy(image_info, priority)
        self.current_strategy = strategy
        
        print(f"\n🔄 Compressing with {strategy.get_name()}...")
        new_size, quality = strategy.compress(image_info)
        
        size_reduction = ((image_info.size_mb - new_size) / image_info.size_mb) * 100
        
        result = {
            "strategy": strategy.get_name(),
            "original_size_mb": image_info.size_mb,
            "compressed_size_mb": new_size,
            "size_reduction_percent": size_reduction,
            "quality_score": quality,
            "lossless": strategy.is_lossless()
        }
        
        print(f"\n📊 Compression Results:")
        print(f"   Size reduction: {size_reduction:.1f}%")
        print(f"   Quality score: {quality}/100")
        print(f"   Lossless: {'Yes' if strategy.is_lossless() else 'No'}")
        
        return result

# Test the advanced strategy selection
print("=== Advanced Strategy Selection - Image Compression ===")

compressor = ImageCompressor()

# Test different image types and priorities
test_images = [
    ImageInfo(1920, 1080, "jpeg", 2.5, has_transparency=False),  # HD photo
    ImageInfo(800, 600, "png", 1.2, has_transparency=True),     # Logo with transparency
    ImageInfo(4000, 3000, "bmp", 15.0, has_transparency=False), # Large uncompressed photo
    ImageInfo(512, 512, "png", 0.8, has_transparency=False),    # Small icon
]

priorities = ["quality", "balanced", "size"]

for i, image in enumerate(test_images, 1):
    print(f"\n{'='*60}")
    print(f"Test Image {i}: {image.width}x{image.height} {image.format.upper()} ({image.size_mb}MB)")
    
    for priority in priorities:
        print(f"\n--- Priority: {priority.upper()} ---")
        result = compressor.compress_image(image, priority)

print("\n🎯 Advanced Strategy Selection Benefits:")
print("1. Automatically chooses optimal compression based on image characteristics")
print("2. Considers user priorities (quality vs size vs balanced)")
print("3. Handles special cases (transparency, large files, etc.)")
print("4. Provides clear reasoning for strategy selection")
print("5. Easy to extend with new strategies and selection rules")

## When to Use (and When NOT to Use)

### ✅ Good Use Cases:

1. **Multiple algorithms** for the same problem (sorting, compression, pathfinding)
2. **Runtime algorithm switching** - choose based on data or user preference
3. **Avoiding conditionals** - replace long if/else chains
4. **A/B testing** - easily switch between different implementations
5. **Plugin architectures** - allow users to provide custom strategies

### ❌ Bad Use Cases (Avoid Strategy When):

1. **Only one algorithm** - don't create strategies for single implementations
2. **Simple variations** - use parameters instead of separate strategies
3. **Algorithms rarely change** - if you'll never switch, don't add the overhead
4. **Performance critical** - strategy pattern adds small overhead

### Example: When NOT to use Strategy

In [ ]:
# ❌ BAD: Over-engineering with Strategy pattern
class AdditionStrategy:
    """DON'T do this - addition is always the same!"""
    
    def add(self, a: int, b: int) -> int:
        return a + b

class Calculator:
    """Unnecessarily complex calculator"""
    
    def __init__(self, addition_strategy: AdditionStrategy):
        self.addition_strategy = addition_strategy  # This is overkill!
    
    def add(self, a: int, b: int) -> int:
        return self.addition_strategy.add(a, b)

# This is unnecessarily complex!
strategy = AdditionStrategy()
calculator = Calculator(strategy)
result = calculator.add(2, 3)
print(f"Overcomplicated result: {result}")

print("\n✅ BETTER: Simple direct implementation")

def add(a: int, b: int) -> int:
    """Simple addition - no strategy needed!"""
    return a + b

# Much simpler and clearer
simple_result = add(2, 3)
print(f"Simple result: {simple_result}")

print("\n🎯 Use Strategy pattern when:")
print("- You have multiple ways to solve the same problem")
print("- Algorithm choice depends on runtime conditions")
print("- You want to eliminate large if/else chains")
print("- You need to support user-defined algorithms")

print("\n🚫 Don't use Strategy pattern when:")
print("- There's only one way to do something")
print("- Variations can be handled with simple parameters")
print("- The algorithm will never change")
print("- Performance overhead matters more than flexibility")

print("\n💡 Better alternatives for simple cases:")
print("- Use function parameters for small variations")
print("- Use enums with switch statements for fixed options")
print("- Use configuration files for simple behavior changes")

## Summary

### Key Takeaways:

1. **Strategy pattern encapsulates algorithms** in interchangeable classes
2. **Eliminates conditional complexity** by replacing if/else chains
3. **Enables runtime algorithm switching** based on context
4. **Promotes loose coupling** between context and algorithms
5. **Easy to extend** with new strategies without changing existing code

### The Strategy Recipe:

```python
# 1. Define strategy interface
class Strategy(ABC):
    @abstractmethod
    def execute(self, data) -> result:
        pass

# 2. Implement concrete strategies
class ConcreteStrategyA(Strategy):
    def execute(self, data):
        # Algorithm A implementation
        pass

class ConcreteStrategyB(Strategy):
    def execute(self, data):
        # Algorithm B implementation
        pass

# 3. Create context that uses strategies
class Context:
    def __init__(self, strategy: Strategy):
        self.strategy = strategy
    
    def set_strategy(self, strategy: Strategy):
        self.strategy = strategy
    
    def execute_strategy(self, data):
        return self.strategy.execute(data)
```

### Remember:
- 🎯 **Use for multiple algorithms** solving the same problem
- 🔄 **Perfect for runtime algorithm switching**
- 🚫 **Don't over-engineer** simple single-algorithm scenarios
- 🧪 **Great for A/B testing** different approaches

**"The Strategy pattern defines a family of algorithms, encapsulates each one, and makes them interchangeable."** 🎯